# Reporte diario promesas realizadas durante el dia

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import date
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/Reportes diarios/Promesas Diarias'
#Definimos la ruta donde estan las asignaciones
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
mes2 = 'Diciembre2020'

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = 3306
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Asignamos valores a los parametros \n",
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y-%H-%M')
today

'20201224'

In [5]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=puerto,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

# Liverpool

In [6]:
#Hacemos la consulta referente a las gestiones de Liverpool
sql_gest_liv = cursor.callproc('liverpool_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
    gestion_liv = pd.DataFrame(result.fetchall())
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']
gestion_liv['credito'] = pd.to_numeric(gestion_liv['credito'])
gestion_liv.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,fecha_promesa,monto_promesa,comentarios
427,9464459,12,2,1400000007326212,VICTOR MANUEL GUERRA REGALADO,5574556931,celular,2020-12-24 08:47:38,RODRIGUEZHECR,RODRIGUEZ HERNANDEZ CRISTIAN,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-24,981.00,5574556931 // PROMESA DE PAGO LIQUIDACIÃN POR...
603,9464063,12,2,1400000007611209,ROSARIO MARIBEL HERNANDEZ SANCHEZ,5530580822,celular,2020-12-24 08:33:19,AGUILARRIAL,AGUILAR RIOS DANIEL ALEJANDRO,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-24,1900.00,5530580822 TT SE COMUNICA PORQUE LE LLEGO UN M...
799,9463614,12,4,13000022296821,JUAN CARLOS CASTILLO OCHOA,2225236144,celular,2020-12-24 08:08:23,OROZCOGOBR,OROZCO GONZALEZ BRENDA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-30,1760.00,fSE AGENDA PAGO DE LIQUIDACION POR $1760 EL D...


In [7]:
#Hacemos la consulta de la asignacion
sql_asig_liv = cursor.callproc('liverpool_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_liv = pd.DataFrame(result.fetchall())
asignacion_liv = asignacion_liv.iloc[:,[0,1,3,24,6,26]]
asignacion_liv.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_liv['credito'] = pd.to_numeric(asignacion_liv['credito'])
asignacion_liv.head(3)

,firma_id,unegocio_id,credito,estado,rfc,division
0,12,0,10000001017274,MEX,MAMA720814JK8,DEPN
1,12,0,10000005443864,TAMPS,HEGD7610029Z4,DEPN
2,12,0,10000005657620,PUE,LADF4609143I1,SER4


In [8]:
#Hacemos el consolidado 
consolidado_liv = pd.merge(gestion_liv,asignacion_liv,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']>50)]
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']<500000)]
consolidado_liv.fillna('COBRANZA',inplace=True)
consolidado_liv.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,fecha_promesa,monto_promesa,comentarios,estado,rfc,division
0,9273958,12,5,13000013194803,SANDIBEL GARCIA ARRIOLA,2296053728,celular,2020-12-04 12:32:42,BUSTOSLUMA,MANUEL BUSTOS LUCERO,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-08,4935.0,2296053728// TT REALIZARA SU SIGUENTE PAGO EL...,VER,GAAS870930EG2,REM4
1,9273488,12,2,1400000006734838,JACOBO FRANCISCO GUTIERREZ,7121544717,celular,2020-12-04 12:11:14,CRUZSAXI,CRUZ SALAZAR XIMENA,PROMESA,ENTRADA WHATS APP,PROMESA DE PAGO LIQUIDACION,2020-12-15,1126.0,"7121544717// tt hace pp para liquidar con $1,1...",MEX,FAGJ840807PR6,SB02
2,9273225,12,1,33000007401079,JOSE ANTONIO TORRES GODINEZ,5513315629,celular,2020-12-04 12:01:24,TONARVAEZMA,TORRES NARVAEZ MAYTE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-30,1883.0,5513315629 TT REALIZARA PAGO LIQUIDACION EL D...,MEX,TOGA690613883,REM2


In [9]:
#Obtenemos datos de las promesas de Liverpool
pivot_liv = pd.pivot_table(consolidado_liv,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv = pd.DataFrame(pivot_liv.to_records())
pivot_liv = pivot_liv.iloc[:,[0,1,4]]
pivot_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_liv.fillna(0,inplace=True)
#pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv

,Division,NumeroPromesas,SumaPromesas
0,REM2,1,1883.0
1,REM4,1,4935.0
2,REM9,1,500.0
3,SB02,4,7348.0
4,pren,1,7000.0
5,All,8,21666.0


In [10]:
#Vemos las promesas reales eliminando duplicados
consolidado_liv2 = consolidado_liv.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_liv2 = pd.pivot_table(consolidado_liv2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv2 = pd.DataFrame(pivot_liv2.to_records())
pivot_liv2 = pivot_liv2.iloc[:,[0,1,4]]
pivot_liv2.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_liv2.fillna(0,inplace=True)
#pivot_liv2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_liv2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv2

,Dictamen,NumeroPromesas,SumaPromesas
0,REM2,1,1883.0
1,REM4,1,4935.0
2,REM9,1,500.0
3,SB02,4,7348.0
4,pren,1,7000.0
5,All,8,21666.0


# Bradesco

In [11]:
#Hacemos la consulta referente a las gestiones de Bradesco
sql_gest_brad = cursor.callproc('bradescard_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
    gestion_brad = pd.DataFrame(result.fetchall())
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']
gestion_brad['credito'] = pd.to_numeric(gestion_brad['credito'])
gestion_brad.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios
92,9275748,1,3,5063690389458010,ARRIAGA ESPINOZA FLOR YANETT,5517520704,celular,2020-12-04 13:39:47,GACHAVEZBR,GARCIA CHAVEZ BRAYAN,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,LLAMADA A TITULAR-PROMESA DE PAGO PARCIAL,2020-12-07,300.00,5517520704 TT REALIZA PAGO POR $300 EL DIA 07...
282,9275381,1,3,4812830085852010,MEX EUAN PILAR DEL ROCIO,9994961529,celular,2020-12-04 13:22:25,LOPEZESMAIG,LOPEZ ESTRADA MA. IGNACIA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,LLAMADA A TITULAR-PROMESA DE PAGO PARCIAL,2020-12-05,500.00,0459994961529 TT HACE PROMESA DE PAGO PARCIAL ...
294,9275319,1,4,4812840717921018,ROJAS GOMEZ SILVIA,7227926269,celular,2020-12-04 13:21:32,GARCIARUAL,GARCIA RUIZ ALEXIS,PROMESA,ENTRADA SMS,PROMESA DE PAGO PARCIAL,ENTRADA SMS-PROMESA DE PAGO PARCIAL,2020-12-05,500.00,7227926269 TT MENCIONA QUE NO PUEDE LIQUIDAR E...


In [12]:
#Convertimos a numero las columnas que necesitemos
gestion_brad[cols] = gestion_brad[cols].apply(pd.to_numeric, errors='coerce')

In [13]:
#Hacemos la consulta de la asignacion
sql_asig_brad = cursor.callproc('bradescard_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_brad = pd.DataFrame(result.fetchall())
asignacion_brad = asignacion_brad.iloc[:,[0,1,3,24,6,26]]
asignacion_brad.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_brad.head(3)

,firma_id,unegocio_id,credito,estado,rfc,division
0,1,3,2869000138080177,YUCATAN,JURM8405047J801,C0 5PV
1,1,3,2869000240445953,VERACRUZ,TOHI650826EZ601,C0 4PV
2,1,3,2869000320452929,PUEBLA,HERL821006J1401,C0 4PV


In [14]:
#Hacemos la union de asignacion y promesas
consolidado_brad = pd.merge(gestion_brad,asignacion_brad,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_brad[cols] = consolidado_brad[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']>50)]
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']<500000)]
consolidado_brad.fillna('COBRANZA',inplace=True)
consolidado_brad.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios,estado,rfc,division
0,9275748,1,3,5063690389458010,ARRIAGA ESPINOZA FLOR YANETT,5517520704,celular,2020-12-04 13:39:47,GACHAVEZBR,GARCIA CHAVEZ BRAYAN,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,LLAMADA A TITULAR-PROMESA DE PAGO PARCIAL,2020-12-07,300.0,5517520704 TT REALIZA PAGO POR $300 EL DIA 07...,CIUDAD DE MÃXICO,AIEF800819AUA01,C0 5PV
1,9275381,1,3,4812830085852010,MEX EUAN PILAR DEL ROCIO,9994961529,celular,2020-12-04 13:22:25,LOPEZESMAIG,LOPEZ ESTRADA MA. IGNACIA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,LLAMADA A TITULAR-PROMESA DE PAGO PARCIAL,2020-12-05,500.0,0459994961529 TT HACE PROMESA DE PAGO PARCIAL ...,YUCATAN,MEEP850708PY801,C0 6PV
2,9275319,1,4,4812840717921018,ROJAS GOMEZ SILVIA,7227926269,celular,2020-12-04 13:21:32,GARCIARUAL,GARCIA RUIZ ALEXIS,PROMESA,ENTRADA SMS,PROMESA DE PAGO PARCIAL,ENTRADA SMS-PROMESA DE PAGO PARCIAL,2020-12-05,500.0,7227926269 TT MENCIONA QUE NO PUEDE LIQUIDAR E...,MÃXICO,ROGS6612238K501,C1


In [15]:
#Obtenemos datos de las promesas de Bradesco
pivot_brad = pd.pivot_table(consolidado_brad,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad = pd.DataFrame(pivot_brad.to_records())
pivot_brad = pivot_brad.iloc[:,[0,1,4]]
pivot_brad.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad.fillna(0,inplace=True)
#pivot_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad

,Division,NumeroPromesas,SumaPromesas
0,C0 4PV,16,30684.0
1,C0 5PV,11,10791.0
2,C0 6PV,7,12367.0
3,C1,14,20476.0
4,C2,4,5445.0
5,C3,5,5100.0
6,C4,1,1440.0
7,All,58,86303.0


In [16]:
#Vemos las promesas reales eliminando duplicados
consolidado_brad2 = consolidado_brad.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_brad2 = pd.pivot_table(consolidado_brad2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad2 = pd.DataFrame(pivot_brad2.to_records())
pivot_brad2 = pivot_brad2.iloc[:,[0,1,4]]
pivot_brad2.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad2.fillna(0,inplace=True)
#pivot_brad2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad2

,Division,NumeroPromesas,SumaPromesas
0,C0 4PV,16,30684.0
1,C0 5PV,11,10791.0
2,C0 6PV,7,12367.0
3,C1,14,20476.0
4,C2,4,5445.0
5,C3,5,5100.0
6,C4,1,1440.0
7,All,58,86303.0


# Credifiel

In [17]:
#Hacemos la consulta referente a las gestiones de Credifiel
sql_gest_cred = cursor.callproc('credifiel_rpt_gestiones_detallado',[today,today])
for result in cursor.stored_results():
    gestion_cred = pd.DataFrame(result.fetchall())
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
cols = ['monto_promesa','unegocio_id']
gestion_cred[cols] = gestion_cred[cols].apply(pd.to_numeric, errors='coerce')
gestion_cred = gestion_cred.loc[gestion_cred['monto_promesa']>0]
gestion_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen
10,9267070,1,2020-12-04,07:25:56,CR-474771,GERALDO VERDUGO ARTURO,6121471521,GONMORALESBM,GONZALEZ MORALES BRENDA MARLEN,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-17,3823.72,"6121471521 TT REALIZARA PAGO 4 17/12 $3,823.72...",PDP,Promesa de Pago,CONTACTO
189,9271902,1,2020-12-04,10:34:28,CR-437255,RASCON TRUJILLO NORMANDO,6623514659,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-05,1000.00,6623514659 TT DARA PRIMER PAGO POR 1000 EL DIA...,PDP,Promesa de Pago,CONTACTO
366,9275731,1,2020-12-04,13:38:02,CR-485694,PEREZ FLORES SANDRA BERENICE,8994224084,ZARZANOJA,ZARZA NORIA JEFREY ALBERTO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,2833.00,"8994224084 TT REALIZARA PAGO 3 $2,833.00 15/12...",PDP,Promesa de Pago,CONTACTO


In [18]:
#Hacemos la consulta de la asignacion
sql_asig_cred = cursor.callproc('credifiel_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_cred = pd.DataFrame(result.fetchall())
asignacion_cred = asignacion_cred.iloc[:,[1,3,32]]
asignacion_cred.columns = ['unegocio_id','credito','division']
#asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_cred.head(3)

,unegocio_id,credito,division
0,1,CR-101922,COBRANZA EXTRAJUDICIAL
1,1,CR-101949,LIQUIDAR
2,1,CR-101952,LIQUIDAR


In [19]:
#Hacemos la union de asignacion y promesas
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',on=['unegocio_id','credito'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']>50)]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred.fillna('COBRANZA',inplace=True)
consolidado_cred.replace('','COBRANZA',inplace=True)
consolidado_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen,division
0,9267070,1,2020-12-04,07:25:56,CR-474771,GERALDO VERDUGO ARTURO,6121471521,GONMORALESBM,GONZALEZ MORALES BRENDA MARLEN,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-17,3823.72,"6121471521 TT REALIZARA PAGO 4 17/12 $3,823.72...",PDP,Promesa de Pago,CONTACTO,EXTRAJUDICIAL
1,9271902,1,2020-12-04,10:34:28,CR-437255,RASCON TRUJILLO NORMANDO,6623514659,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-05,1000.00,6623514659 TT DARA PRIMER PAGO POR 1000 EL DIA...,PDP,Promesa de Pago,CONTACTO,EXTRAJUDICIAL
2,9275731,1,2020-12-04,13:38:02,CR-485694,PEREZ FLORES SANDRA BERENICE,8994224084,ZARZANOJA,ZARZA NORIA JEFREY ALBERTO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,2833.00,"8994224084 TT REALIZARA PAGO 3 $2,833.00 15/12...",PDP,Promesa de Pago,CONTACTO,COBRANZA EXTRAJUDICIAL


In [20]:
#Obtenemos datos de las promesas de Credifiel
pivot_cred = pd.pivot_table(consolidado_cred,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_cred = pd.DataFrame(pivot_cred.to_records())
pivot_cred = pivot_cred.iloc[:,[0,3,2]]
pivot_cred.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_cred.fillna(0,inplace=True)
pivot_cred = pivot_cred.loc[pivot_cred['SumaPromesas']>0]
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred

,Division,NumeroPromesas,SumaPromesas
0,COBRANZA EXTRAJUDICIAL,1,2833.00
1,EXTRAJUDICIAL,2,4823.72


In [21]:
#Vemos las promesas reales eliminando duplicados
consolidado_cred2 = consolidado_cred.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_cred2 = pd.pivot_table(consolidado_cred2,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_cred2 = pd.DataFrame(pivot_cred2.to_records())
pivot_cred2 = pivot_cred2.iloc[:,[0,3,2]]
pivot_cred2.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_cred2.fillna(0,inplace=True)
pivot_cred2 = pivot_cred2.loc[pivot_cred2['SumaPromesas']>0]
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred2

,Division,NumeroPromesas,SumaPromesas
0,COBRANZA EXTRAJUDICIAL,1,2833.00
1,EXTRAJUDICIAL,2,4823.72


# Banco Azteca

In [22]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
sql_gest_baz = cursor.callproc('baz_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
     gestion_baz = pd.DataFrame(result.fetchall())
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
gestion_baz.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion_resultado,fecha_promesa,monto_promesa,comentarios
367,9274973,7,1,1-1-5063-18811,MARIA DEL CARMEN AZCONA SALINAS,5535642252,celular,2020-12-04 10:43:30,VIEYRAITTR,TREJO VIEYRA ITZEL YANET,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-08,3016.00,5535642252 PRE INDICA PAGO INICIAL DE $ 3016 ...
368,9274971,7,1,1-1-6027-51703,LAURA MANUELA GARCIA CORREA,5567599408,celular,2020-12-04 12:21:10,VIEYRAITTR,TREJO VIEYRA ITZEL YANET,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-08,1828.00,5567599408 5618612054 PRE CONVENIO DE PAGO SO...
637,9274268,7,2,1-1-4221-38328,RICARDO LOPEZ MURILLO,3321993686,celular,2020-12-04 12:05:49,LEONSASU,LEON SALINAS SUSANA,PROMESA,ENTRADA WHATSAPP-PROMESA DE PAGO PARCIAL,2020-12-11,580.00,3321993686 SE CONTACTA TT VIA WHAT REPORTANDO ...


In [23]:
#new_asignaciones = os.listdir('/home/estadistico/Documents/Erick/Banco Azteca/Asignacion')
#sample_cols_to_keep =['#FIIDCAMPANA', 'FIPAIS', 'FICANAL','FISUCURSAL','FIFOLIO','FISEMATRASMAX','FNSALDO','FNSALDOCAPITAL','FNPAGOREQ','FCTEL1','FCTIPO1','FNDIA_PAGO','FIDIASATRASOMAX','FNCAPPAGODISP','FNABONOSEMANAL','FNCAPACIDADPAGO','FILCRACTIVA','FDFECPROXPAG','FNTASAINT','CP','fcnombreos','fcappaternoos','FITERRITORIO','FCDESCTERRITORIO','FIZONA','FCDESCZONA','FIREGION','FCDESCREGION','FIGERENCIA','FCDESCGERENCIA','FCBESTTIMETOCALL','FECHA']
#df_iter = pd.read_csv('/home/estadistico/Documents/Erick/Banco Azteca/Asignacion_csv/Asignacion banco azteca '+mes+'.csv', chunksize=20000, usecols=sample_cols_to_keep) 
#df_lst = [] 
#for df_ in df_iter: 
#            tmp_df = (df_.rename(columns={col: col.lower() for col in df_.columns}))
#            df_lst += [tmp_df.copy()] 
#asignacion_baz = pd.concat(df_lst)

In [24]:
asignacion_baz = pd.read_csv(''+ruta_asig_baz+'/Asignacion banco azteca '+str(mes2)+'.csv')
asignacion_baz = asignacion_baz.rename(columns=str.lower)
asignacion_baz.head(3)

,#fiidcampana,fipais,ficanal,fisucursal,fifolio
0,164,1,1,5496,4183
1,164,1,1,110,52389
2,164,1,1,5254,28682


In [25]:
asignacion_baz['credito'] = asignacion_baz['fipais'].map(str)+'-'+asignacion_baz['ficanal'].map(str)+'-'+asignacion_baz['fisucursal'].map(str)+'-'+asignacion_baz['fifolio'].map(str)
asignacion_baz.head(3)

,#fiidcampana,fipais,ficanal,fisucursal,fifolio,credito
0,164,1,1,5496,4183,1-1-5496-4183
1,164,1,1,110,52389,1-1-110-52389
2,164,1,1,5254,28682,1-1-5254-28682


In [26]:
consolidado_baz = pd.merge(gestion_baz,asignacion_baz,how='left',on='credito')
cols = ['monto_promesa','#fiidcampana']
consolidado_baz[cols] = consolidado_baz[cols].apply(pd.to_numeric, errors='coerce')
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']>50)]
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']<500000)]
consolidado_baz.fillna('COBRANZA',inplace=True)
consolidado_baz.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion_resultado,fecha_promesa,monto_promesa,comentarios,#fiidcampana,fipais,ficanal,fisucursal,fifolio
0,9274973,7,1,1-1-5063-18811,MARIA DEL CARMEN AZCONA SALINAS,5535642252,celular,2020-12-04 10:43:30,VIEYRAITTR,TREJO VIEYRA ITZEL YANET,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-08,3016.0,5535642252 PRE INDICA PAGO INICIAL DE $ 3016 ...,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA
1,9274971,7,1,1-1-6027-51703,LAURA MANUELA GARCIA CORREA,5567599408,celular,2020-12-04 12:21:10,VIEYRAITTR,TREJO VIEYRA ITZEL YANET,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-08,1828.0,5567599408 5618612054 PRE CONVENIO DE PAGO SO...,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA
2,9274268,7,2,1-1-4221-38328,RICARDO LOPEZ MURILLO,3321993686,celular,2020-12-04 12:05:49,LEONSASU,LEON SALINAS SUSANA,PROMESA,ENTRADA WHATSAPP-PROMESA DE PAGO PARCIAL,2020-12-11,580.0,3321993686 SE CONTACTA TT VIA WHAT REPORTANDO ...,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA


In [27]:
#Obtenemos datos de las promesas de Banco Azteca
pivot_baz = pd.pivot_table(consolidado_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_baz = pd.DataFrame(pivot_baz.to_records())
pivot_baz = pivot_baz.iloc[:,[0,3,2]]
pivot_baz.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_baz.fillna(0,inplace=True)
#pivot_baz[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_baz[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_baz

,Dictamen,NumeroPromesas,SumaPromesas
0,PROMESA,28,22632.0


In [28]:
#Vemos las promesas reales eliminando duplicados
consolidado_baz2 = consolidado_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_baz2 = pd.pivot_table(consolidado_baz2,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_baz2 = pd.DataFrame(pivot_baz2.to_records())
pivot_baz2 = pivot_baz2.iloc[:,[0,3,2]]
pivot_baz2.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_baz2.fillna(0,inplace=True)
#pivot_baz2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_baz2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_baz2

,Dictamen,NumeroPromesas,SumaPromesas
0,PROMESA,28,22632.0


In [29]:
writer = pd.ExcelWriter(''+ruta+'/Reporte Promesas Diarias '+now+'.xlsx', engine='xlsxwriter')
pivot_liv.to_excel(writer,'PromDiariaLivBruto',index=False,header=True)
pivot_liv2.to_excel(writer,'PromDiariaLivReal',index=False,header=True)
pivot_brad.to_excel(writer,'PromDiariaBradBruto',index=False,header=True)
pivot_brad2.to_excel(writer,'PromDiariaBradReal',index=False,header=True)
pivot_cred.to_excel(writer,'PromDiariaCredBruto',index=False,header=True)
pivot_cred2.to_excel(writer,'PromDiariaCredReal',index=False,header=True)
pivot_baz.to_excel(writer,'PromDiariaBAZBruto',index=False,header=True)
pivot_baz2.to_excel(writer,'PromDiariaBAZReal',index=False,header=True)

writer.save()
writer.close()